# # Table of Contents
# 1. [Importación de Librerías](#import-librerias)
# 2. [Análisis Exploratorio Avanzado (EDA)](#eda-avanzado)
# 3. [Correlaciones, Boxplots e Histogramas](#plots)
# 4. [Categorical vs. Salary](#cat-vs-salary)
# 5. [Scatter Experience vs. Age vs. Salary](#scatter)

# # Importación de Librerías <a id="import-librerias"></a>

In [17]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Optional, List
from project_pwc.config import FIGURES_DIR

sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
num_cols = ["Age", "Years of Experience", "Salary"]

# # Análisis Exploratorio Avanzado (EDA) <a id="eda-avanzado"></a>

In [4]:
df_clean = pd.read_csv('C:/Users/Usuario/Documents/prueba_pwc/predictive_salary_model/data/interim/dataset_cleaned.csv')

In [3]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   373 non-null    int64  
 1   Age                  373 non-null    int64  
 2   Gender               373 non-null    object 
 3   Education Level      373 non-null    object 
 4   Job Title            373 non-null    object 
 5   Years of Experience  373 non-null    int64  
 6   Salary               373 non-null    float64
 7   Description          373 non-null    object 
dtypes: float64(1), int64(3), object(4)
memory usage: 23.4+ KB


# # Correlaciones, Boxplots e Histogramas <a id="plots"></a>

In [31]:
def plot_correlation_heatmap(df: pd.DataFrame, numerical_cols: List[str], filename: str = "correlation_heatmap.png") -> None:
    """
    Genera y muestra un heatmap de correlaciones entre columnas numéricas,
    guardando la figura en reports/figures/.
    """
    corr_matrix = df[numerical_cols].corr()

    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlation Heatmap")
    
    output_path = os.path.join(FIGURES_DIR, filename)
    plt.savefig(output_path)
    print(f"Heatmap guardado en: {output_path}")

plot_correlation_heatmap(df_clean, num_cols)

Descripción

Se muestra la correlación entre Age, Years of Experience y Salary.
Observamos que Age y Years of Experience tienen una correlación muy alta (~0.98), y ambas se relacionan fuertemente con Salary (~0.92 y ~0.93, respectivamente).
Conclusiones

Este nivel de correlación sugiere multicolinealidad: en un modelo lineal, podría ocasionar inestabilidad en los coeficientes.
Las tres variables son muy relevantes para explicar la variación de Salary, pero tal vez sea suficiente con una sola de ellas (Age o Experience) en modelos lineales.
En algoritmos de árboles (RandomForest, XGBoost) esta redundancia impacta menos, pero es bueno tenerlo en mente.

In [ ]:
def plot_boxplots(df: pd.DataFrame, numerical_cols: List[str], prefix: str = "boxplot") -> None:

    for col in numerical_cols:
        plt.figure(figsize=(5, 4))
        sns.boxplot(x=df[col])
        plt.title(f"Boxplot for {col}")
        
        filename = f"{prefix}_{col}.png"
        output_path = os.path.join(FIGURES_DIR, filename)
        plt.savefig(output_path)
        print(f"Boxplot de {col} guardado en: {output_path}")

plot_boxplots(df_clean, num_cols)

Descripción

Cada boxplot revela la distribución y posibles outliers:
Age: Rango 23–53, mediana cerca de 36, sin outliers extremos.
Years of Experience: Rango 0–25, picos en 5–10 años y casos puntuales de 20+ años.
Salary: Rango ~350–250,000, con una mediana ~95,000 y valores altos que superan los 200,000.
Conclusiones

Age no presenta valores anómalos.
Experience puede tener saltos marcados (nuevos vs. muy experimentados).
Salary está sesgado a la derecha; hay pocos valores muy altos que podrían considerarse outliers en modelos lineales.

In [ ]:
def plot_histograms(df: pd.DataFrame, numerical_cols: List[str], prefix: str = "histogram") -> None:

    for col in numerical_cols:
        plt.figure(figsize=(5, 4))
        sns.histplot(df[col], kde=True, color="teal", bins=20)
        plt.title(f"Distribution of {col}")

        filename = f"{prefix}_{col}.png"
        output_path = os.path.join(FIGURES_DIR, filename)
        plt.savefig(output_path)
        print(f"Histograma de {col} guardado en: {output_path}")

plot_histograms(df_clean, num_cols)

Descripción

Age: Aproximación casi normal, con mayor densidad entre 30–40 años.
Years of Experience: Distribución más irregular; un pico en los primeros años (0–5) y otro alrededor de ~15.
Salary: Marcado sesgo a la derecha (long tail); el grupo principal se ubica entre 50,000 y 120,000, pero también hay salarios de hasta 250,000.
Conclusiones

Con una distribución tan asimétrica en Salary, podrías aplicar una transformación log para métodos como Regresión Lineal.
Se confirma la idea de crear “bins” para Experience (junior, semi-senior, senior, etc.) dada su distribución no uniforme.

# # Categorical vs. Salary <a id="cat-vs-salary"></a>

In [ ]:
def analyze_categorical_vs_salary(df: pd.DataFrame, cat_col: str, salary_col: str = "Salary") -> None:

    mean_salary_by_cat = df.groupby(cat_col)[salary_col].mean().sort_values(ascending=False)
    print(f"=== Mean {salary_col} by {cat_col} ===\n{mean_salary_by_cat}\n")

    plt.figure(figsize=(8, 4))
    sns.violinplot(data=df, x=cat_col, y=salary_col, palette="viridis")
    plt.title(f"{salary_col} distribution by {cat_col}")
    
    output_filename = f"violin_{cat_col}.png"
    output_path = os.path.join(FIGURES_DIR, output_filename)
    plt.savefig(output_path)
    print(f"Violinplot guardado en: {output_path}")

analyze_categorical_vs_salary(df_clean, "Education Level")

* Gender: El violinplot sugiere diferencias en la mediana y los valores extremos de salario (male vs. female vs. missing).
* Education Level: El salario tiende a ser mayor conforme aumenta la formación académica (PhD > Master’s > Bachelor’s), aunque con solapamientos entre categorías.

Implicaciones:
Ambas variables categóricas (Gender, Education Level) podrían tener poder predictivo.
El tratamiento de “Missing” en ambos casos (género o nivel educativo) es importante; conviene mantenerlo como una categoría adicional para no perder registros.
Se recomienda One-Hot Encoding u otras estrategias de codificación que reflejen el nivel formativo y el género en el modelado.

# # Scatter Experience vs. Age vs. Salary <a id="scatter"></a>

In [ ]:
def scatter_experience_age_salary(df: pd.DataFrame,
                                  x_col: str = "Age",
                                  y_col: str = "Years of Experience",
                                  hue_col: Optional[str] = "Gender",
                                  size_col: Optional[str] = "Salary",
                                  filename: str = "scatter_experience_age_salary.png") -> None:

    plt.figure(figsize=(8, 5))
    sns.scatterplot(data=df, x=x_col, y=y_col, hue=hue_col, size=size_col, sizes=(20, 200), alpha=0.7)
    plt.title(f"{y_col} vs {x_col} (hue={hue_col}, size={size_col})")
    
    output_path = os.path.join(FIGURES_DIR, filename)
    plt.savefig(output_path)
    print(f"Scatterplot guardado en: {output_path}")

scatter_experience_age_salary(df_clean)

Descripción

Se aprecia una relación casi lineal: a mayor edad, mayor experiencia, lo que confirma la fuerte correlación numérica.
Los puntos grandes (representando salarios altos) suelen estar en la zona de mayor experiencia y/o edad, aunque hay excepciones.
El color sugiere que tanto hombres como mujeres se reparten a lo largo de toda la diagonal, con variaciones notables en el tamaño (Salary).
Conclusiones

Es un claro indicio de multicolinealidad entre edad y experiencia.
Un alto Salary suele coincidir con un mayor rango de experiencia/edad, pero también hay casos medianos o altos en rangos intermedios.
Podrías eliminar una de las dos variables en modelos lineales o crear una tercera (por ejemplo, Age - YearsExperience) si deseas capturar la idea de “edad de inicio profesional”.